# Procena cene kuća pomoću nerunonske mreže

## Opis problema i cilj projekta
Cilj ovog projekta je razviti regresioni model zasnovan na neuronskoj mreži koji može precizno da proceni cenu kuća na osnovu njihovih karakteristika (poput kvadrature, broja soba, godine izgradnje…).  
Ciljno obeležje je `SalePrice`, odnosno prodajna cena izražena u američkim dolarima (USD).

## Faze izrade modela

Proces rešavanja problema procene cena kuća obuhvata sledeće faze:

1. **Priprema podataka**
   - Učitavanje skupa podataka (`Ames Housing Dataset`).
   - Uklanjanje kolona i zapisa sa previše nedostajućih vrednosti.
   - Imputacija nedostajućih vrednosti: numeričke zamenjene medijanom, a kategorijske vrednosti su zamenjene specijalnom oznakom ('Missing') pre nego što su enkodirane u numerički oblik.
   - Kodiranje kategorijskih atributa u numeričke vrednosti (one-hot encoding).
   - Analiza korelacije atributa sa ciljnim obeležjem (`SalePrice`) i uklanjanje nerelevantnih atributa.

2. **Smanjenje dimenzionalnosti**
   - Primena PCA (Principal Component Analysis) kako bi se redukovala dimenzionalnost i transformisali atributi u nove komponente uz očuvanje varijanse.

3. **Podela podataka**
   - Deljenje podataka na trening, validacioni i test skup (70:15:15).

4. **Treniranje modela**
   - Izgradnja i treniranje neuronske mreže za regresiju.
   - Korišćenje MSE (Mean Squared Error) funkcije gubitka i Adam optimizatora.
   - Pratimo performanse na validacionom skupu i primenjujemo early stopping.

5. **Evaluacija i testiranje**
   - Ocena modela na test skupu (MSE, RMSE, MAE, R2).
   - Vizuelizacija grešaka i poređenje predviđenih i stvarnih cena kuća.


## Koraci u razvoju modela

#### 1. Učitavanje skupa podataka
Koristimo `pandas.read_csv()` da učitamo dataset `AmesHousing.csv`.

#### 2. Uklanjanje kolona sa previše nedostajućih vrednosti
Funkcija `drop_high_missing(df, threshold)` uklanja sve kolone u kojima je više od `threshold` (30%) vrednosti **NaN**.  
Ovo smanjuje uticaj nepotpunih podataka na model.

#### 3. Uklanjanje atributa sa slabom korelacijom sa ciljem
Funkcija `remove_low_correlation_features(df, target_col, threshold)` uklanja numeričke kolone čija **apsolutna korelacija** sa ciljnim obeležjem (`SalePrice`) je manja od zadate vrednosti (`threshold`, npr. 0.05).

- Zadržavamo samo relevantne atribute koji su statistički povezani sa cenom.
- Funkcija takođe generiše vizualni prikaz zadržanih i uklonjenih atributa (grafik sa barovima).

In [11]:
import pandas as pd
from data.preprocessing import (
    drop_high_missing,
    remove_low_correlation_features
)
data_path = "dataset/AmesHousing.csv"
drop_missing_thresh = 0.30
correlation_thresh = 0.05
target = "SalePrice"
output_dir = "output"

# 1) Load data
df = pd.read_csv(data_path)
print("Loaded dataset shape:", df.shape)

# 2) Drop columns with too many missing values
df = drop_high_missing(df, threshold=0.30)
print("After dropping columns:", df.shape)

# 3) Drop columns with correlation less then given
if correlation_thresh > 0:
    df = remove_low_correlation_features(df, target_col=target, 
                                        threshold=correlation_thresh,
                                        output_dir=output_dir)
    print("After removing low-correlation features:", df.shape)

Loaded dataset shape: (2930, 82)
Dropping 6 columns with >30% missing values.
After dropping columns: (2930, 76)
Removing 8 features with correlation < 0.05
After removing low-correlation features: (2930, 68)


### Vizualni prikaz zadržanih i uklonjenih atributa

![Correlation Analysis](presentation%20files/correlation_analysis.png)

#### 4. Podela skupa podataka na trening, validacioni i test skup

U ovom koraku delimo podatke na ulazne karakteristike (`X`) i ciljnu promenljivu (`y`), a zatim ih raspoređujemo u tri skupa:

1. **Trening skup (train set)** – koristi se za treniranje modela.
2. **Validacioni skup (val set)** – koristi se tokom treniranja za podešavanje hiperparametara i early stopping.
3. **Test skup (test set)** – koristi se **samo na kraju** za evaluaciju konačnog modela.

---

#### Detalji:
- Prvo izdvajamo test skup (`test_size`, 15%) iz celokupnog skupa podataka.
- Preostali podaci (`X_temp`) se dele na trening i validacioni skup prema proporciji `val_size` u odnosu na ukupni skup.
- Na kraju proveravamo veličine svih skupova da bismo se uverili da je podela ispravna.


In [12]:
from data.preprocessing import (
    split_features_target
)
from sklearn.model_selection import train_test_split
test_size = 0.15
val_size = 0.15
random_state = 42

# 4) split X/y and then train/val/test (we must split BEFORE fitting preprocessors)
X, y = split_features_target(df, target_col=target)
# First split off test
test_size = test_size
val_size = val_size
# Combine val+test fraction relative splitting
X_temp, X_test, y_temp, y_test = train_test_split(
    X, y, test_size=test_size, random_state=random_state
)
# Now split X_temp into train and val
# val fraction of original = val_size -> fraction of X_temp = val_size / (1 - test_size)
val_fraction_of_temp = val_size / (1.0 - test_size)
X_train, X_val, y_train, y_val = train_test_split(
    X_temp, y_temp, test_size=val_fraction_of_temp, random_state=random_state
)

print(f"Sizes -> train: {len(X_train)}, val: {len(X_val)}, test: {len(X_test)}")

Sizes -> train: 2050, val: 440, test: 440


#### 5. Fitovanje predprocesora na trening skupu

U ovom koraku pripremamo sve transformacije podataka koristeći **samo trening skup**, kako bismo izbegli curenje informacija iz validacionog ili test skupa.  

- **Numeričke kolone**: nedostajući podaci se zamenjuju **medijanom**.  
- **Kategorijske kolone**: nedostajuće vrednosti se zamenjuju tokenom `"Missing"`, a zatim se kolone enkodiraju koristeći **One-Hot Encoding**.  
- **Skaliranje numeričkih kolona**: primenjuje se `StandardScaler` (mean=0, std=1) da bi mreža lakše učila.  
- Rezultat je objekat `preprocessors` koji sadrži sve fitovane transformacije i liste kolona.

#### 6. Transformacija skupova podataka

Koristeći fitovane predprocesore, transformišemo sve skupove podataka (train, val, test):

1. Numeričke kolone se impute-uju i skaliraju prema parametrima iz trening skupa.  
2. Kategorijske kolone se enkodiraju i impute-uju `"Missing"` gde je potrebno.  
3. Sve kolone se kombinuju u **jedan numpy niz** spreman za neuronsku mrežu (`X_train_np`, `X_val_np`, `X_test_np`).  

- Rezultat: mreža dobija **numerički i normalizovan skup podataka**, spreman za treniranje i evaluaciju.

In [13]:
from data.preprocessing import (
    build_preprocessors,
    transform_dataframe,
)

# 5) Fit preprocessors on train only
preprocessors = build_preprocessors(X_train)

# 6) Transform datasets
X_train_np = transform_dataframe(X_train, preprocessors)
X_val_np = transform_dataframe(X_val, preprocessors)
X_test_np = transform_dataframe(X_test, preprocessors)

Numeric cols: 30, Categorical cols: 37


#### 7. Primena PCA (Principal Component Analysis)

U ovom koraku vršimo **redukciju dimenzionalnosti** koristeći PCA, što pomaže da se:
- Smanji broj ulaznih atributa (feature-a) dok se zadržava što više informacija.  
- Smanji rizik od **overfitting-a** i ubrza treniranje mreže.  

**Postupak:**
1. Kreiramo PCA objekat sa brojem komponenti `args.pca_components`.  
2. Fitujemo PCA **samo na trening skupu** (`X_train_np`).  
3. Transformišemo sve skupove (train, val, test) u novi prostor glavnih komponenti.  
4. Novi dimenzionalitet podataka (`X_train_np.shape[1]`) odgovara broju PCA komponenti.  
5. Vizualizujemo **doprinos originalnih atributa svakoj PCA komponenti** koristeći `plot_pca_feature_contributions`.  

- Ovim korakom zadržavamo najinformativnije kombinacije atributa dok uklanjamo redundantne ili slabo relevantne.


In [14]:
from sklearn.decomposition import PCA
from data.preprocessing import plot_pca_feature_contributions
pca_components = 25

# 7) PCA
if pca_components and pca_components > 0:
    pca = PCA(n_components=pca_components, random_state=random_state)
    pca.fit(X_train_np)
    X_train_np = pca.transform(X_train_np)
    X_val_np = pca.transform(X_val_np)
    X_test_np = pca.transform(X_test_np)
    print(f"PCA applied: new feature dim {X_train_np.shape[1]}")
else:
    print(f"No PCA. Feature dim: {X_train_np.shape[1]}")

all_feature_names = preprocessors["numeric_cols"] + list(preprocessors["onehot_encoder"].get_feature_names_out(preprocessors["categorical_cols"]))
plot_pca_feature_contributions(pca, all_feature_names, output_dir, top_n=10)

PCA applied: new feature dim 25
PCA contribution plots saved to output


#### Analiza doprinosa atributa PCA komponentama

Funkcija `plot_pca_feature_contributions` generiše **heatmap** koja prikazuje težine (loadings) svakog atributa u svakoj PCA komponenti:

- **Redovi**: originalni atributi (npr. kvadratura, broj soba, godina izgradnje…).  
- **Kolone**: PCA komponente (PC1, PC2, …).  
- **Boje**: vrednost težine (loading) – crveno označava pozitivni doprinos, plavo negativni.  

Parametar `top_n=10` prikazuje samo 10 atributa sa najvećim apsolutnim doprinosom po komponenti, čime se heatmap čini preglednijim.

![PCA Loadings Heatmap](presentation%20files/pca_loadings_heatmap.png)

#### 8. Kreiranje DataLoader-a

U ovom koraku pripremamo podatke za treniranje neuronske mreže koristeći **PyTorch DataLoader-e**:

1. **TabularDataset**  
   Kreiramo custom dataset za tabelarne podatke (`TabularDataset`), koji kombinuje ulazne karakteristike (`X_train_np`, `X_val_np`, `X_test_np`) i ciljne vrednosti (`y_train`, `y_val`, `y_test`).

2. **DataLoader**  
   - `train_loader` – koristi se za treniranje. Podešen sa `shuffle=True` da bi se podaci nasumično mešali svakom epohom.  
   - `val_loader` i `test_loader` – koriste se za validaciju i testiranje, `shuffle=False` kako bi redosled podataka bio konzistentan.  
   - `batch_size` određuje veličinu batch-a za treniranje i evaluaciju.

DataLoader omogućava efikasno učitavanje podataka po batch-evima i olakšava treniranje neuronske mreže na velikim skupovima podataka.

In [15]:
from data.dataset import TabularDataset
from torch.utils.data import DataLoader
batch_size = 64

train_ds = TabularDataset(X_train_np, y_train)
val_ds = TabularDataset(X_val_np, y_val)
test_ds = TabularDataset(X_test_np, y_test)

train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_ds, batch_size=batch_size, shuffle=False)

#### 9. Kreiranje modela

U ovom koraku definišemo neuronsku mrežu za regresiju koristeći **PyTorch**:

- **RegressionNet** je potpuno povezani (fully connected) mrežni model koji prima ulazne karakteristike i predviđa kontinualnu vrednost (cenu kuće).  
- Model se sastoji od niza linearnih slojeva (`nn.Linear`) sa **ReLU** aktivacijom i opcionim **Dropout** slojevima za regularizaciju.  
- Broj skrivenih slojeva i veličina svakog sloja definišu se listom `hidden_layers`, a `dropout` određuje verovatnoću isključivanja neurona tokom treninga.

#### 10. Treniranje modela

Model se trenira koristeći funkciju `train_model`:

1. **Loss funkcija:** `MSELoss` (Mean Squared Error) meri prosečnu kvadratnu grešku između predviđenih i stvarnih vrednosti. 
2. **Optimizer:** `Adam` sa zadatim `learning rate` i opcionalnim `weight_decay` (L2 regularizacija).  
3. **Early stopping:** prateći validacioni gubitak (`val_loss`), treniranje se prekida ako se gubitak ne poboljšava `patience` epohama.  
4. **Prikupljanje statistika:** lista `train_losses` i `val_losses` čuva gubitke po epohi za kasniju vizualizaciju.  

**Proces treniranja po epohi:**
- Model se postavlja u `train` režim.
- Svaki batch prolazi kroz mrežu (`forward pass`) i računa se gubitak.
- Optimizator ažurira težine (`backward pass` + `optimizer.step()`).
- Nakon prolaska kroz sve batch-eve, računa se prosečni gubitak za epohu.
- Model se zatim evaluira na validacionom skupu (`eval` režim) da bi se pratila generalizacija.
- Ako je validacioni gubitak najbolji do sada, model se čuva kao najbolji.
- Ukoliko broj epoha odredjen parametrom **patience** nije pokazao napredovanje kod vrednosti validacione loss funckije - **primenjuje se early stopping**

Na kraju, vraća se trenirani model sa najboljim težinama i liste gubitaka za treniranje i validaciju.


In [16]:
import torch
from models.neural_net import RegressionNet
from training.training import (
    train_model
)
hidden_layers = [512, 256, 128]
dropout = 0.0
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
epochs = 500
lr = 0.02
patience = 20

# 9) Create model
input_dim = X_train_np.shape[1]
model = RegressionNet(input_dim=input_dim, hidden_layers=hidden_layers, dropout=dropout)
print("Model:", model)

# 10) Train
model, train_losses, val_losses = train_model(
    model, device, train_loader, val_loader,
    epochs=epochs, lr=lr, patience=patience
)

Model: RegressionNet(
  (net): Sequential(
    (0): Linear(in_features=25, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=256, bias=True)
    (3): ReLU()
    (4): Linear(in_features=256, out_features=128, bias=True)
    (5): ReLU()
    (6): Linear(in_features=128, out_features=1, bias=True)
  )
)
Epoch 025: train_loss=698344358.4624, val_loss=1058779447.8545
Epoch 050: train_loss=489729998.2595, val_loss=871578612.3636
Epoch 075: train_loss=382856448.3122, val_loss=919048960.0000
Early stopping on epoch 78. Best val loss: 783799740.509091


#### 11. Vizualizacija treniranog gubitka (Loss)

Na slici je prikazan tok gubitka (MSE) kroz epohe za **trening** i **validacioni** skup.  
Možemo pratiti konvergenciju modela i eventualni overfitting.

![Loss curve](presentation%20files/loss_curve.png)

#### 12. Evaluacija modela na test skupu

Nakon treniranja, model je evaluiran na **test skupu** koji nije korišćen ni za treniranje ni za validaciju.  
Koristimo standardne metrike za regresiju kako bismo procenili performanse modela:

- **MSE (Mean Squared Error)** – 768,225,664  
  Prosečna kvadratna greška između stvarnih i predviđenih vrednosti. Veća vrednost označava veća odstupanja pojedinačnih predikcija.

- **RMSE (Root Mean Squared Error)** – 27,716.88  
  Koren MSE, intuitivniji prikaz greške u istim jedinicama kao i ciljna promenljiva (`SalePrice`). Ovo znači da je prosečna greška modela oko $27,700 po kući.

- **MAE (Mean Absolute Error)** – 18,201.21  
  Prosečna apsolutna greška između predikcija i stvarnih vrednosti. Manja od RMSE, što sugeriše da ekstremni outlieri povećavaju kvadratnu grešku.

- **R² (Coefficient of Determination)** – 0.90  
  Ova vrednost pokazuje da model objašnjava oko 90% varijanse prodajne cene (`SalePrice`). To je veoma dobar rezultat, ukazujući na to da model precizno predviđa većinu promena u cenama kuća.

**Zaključak:**  
Model je postigao visoku tačnost i dobro generalizuje na nove podatke. Iako postoje pojedinačne greške koje utiču na MSE, ukupna sposobnost modela da predvidi cenu kuća je vrlo dobra, što potvrđuje visok R² skor.

In [17]:
from training.training import evaluate_model

# 12) Evaluate on test
test_metrics = evaluate_model(model, device, test_loader)
print("Test metrics:")
print(f"  MSE:  {test_metrics['mse']:.4f}")
print(f"  RMSE: {test_metrics['rmse']:.4f}")
print(f"  MAE:  {test_metrics['mae']:.4f}")
print(f"  R²:   {test_metrics['r2']:.4f}")

Test metrics:
  MSE:  768225664.0000
  RMSE: 27716.8848
  MAE:  18201.2148
  R²:   0.9000


#### 13. Scatter plot – Stvarne vs Predviđene vrednosti

Na ovom scatter plot-u prikazujemo stvarne vrednosti cene kuća (`True SalePrice`) u odnosu na vrednosti predviđene modelom (`Predicted SalePrice`) na test skupu.

- Svaka tačka predstavlja jednu kuću iz test skupa.
- Isprekidana linija (y = x) prikazuje idealnu situaciju gde su predviđene cene kuća tačno jednake stvarnim cenama.
- Tačke blizu linije označavaju tačne predikcije, dok tačke udaljene od linije predstavljaju greške u predikciji.

Ovaj graf vizualno potvrđuje sposobnost modela da precizno predviđa cene kuća.

![True vs Predicted](presentation%20files/true_vs_pred.png)

#### 14. Grafik važnosti komponenti

U ovom koraku prikazujemo koliko svaka komponenta (npr. PCA komponenta) doprinosi modelu:
- Visina svakog bara predstavlja prosečnu apsolutnu vrednost težina prve linearne sloja neuronske mreže za datu komponentu.
- Komponente sa većim vrednostima imaju veći uticaj na predviđanje cena kuća.
- Ovaj grafikon pomaže da identifikujemo koje komponente model najviše koristi i koje karakteristike podataka su najinformativnije.

![True vs Predicted](presentation%20files/pca_component_importance.png)

#### 15. Analiza grešaka i reziduala

U ovom koraku vizualizujemo performanse modela kroz raspodelu grešaka:
- Greške (errors) predstavljaju razliku između stvarne i predviđene cene kuće (true - pred).
- Reziduali pokazuju koliko su predikcije odstupile od stvarnih vrednosti.
- Ova analiza omogućava uvid u to da li model sistematski precenjuje ili potcenjuje cene, kao i da li postoje ekstremni odmaknuti slučajevi.

Važno je uočiti da greške mogu biti uzorak **normalne raspodele**. To može da znači nekoliko stvari:
1. **Model je dobro kalibrisan** – predikcije su, u proseku, tačne, i odstupanja su raspoređena simetrično oko nule.
2. **Nema sistematske pristrasnosti** – model ne precenjuje ni potcenjuje konstantno; greške su slučajne i ne pokazuju obrazac.
3. **Validnost klasičnih statističkih metoda** – normalna raspodela grešaka omogućava primenu metoda kao što su intervali poverenja, t-testovi i ANOVA, jer su osnovni preduslovi ispunjeni.
4. **Predikcija varijanse** – većina grešaka je mala, a ekstremne vrednosti (outlajeri) su retke, što olakšava interpretaciju i procenu rizika.

![True vs Predicted](presentation%20files/error_analysis.png)

## Hyperparameter tuning (Grid search)

Dodatan korak u razvoju projekta uključuje **grid search** - vršimo eksperimente sa različitim kombinacijama hiperparametara neuronske mreže:
- **pca_componentts**: Broj glavnih komponenti nakon PCA redukcije dimenzionalnosti. Veći broj zadržava više informacija, manji broj smanjuje kompleksnost i vreme treniranja.
- **lr**: Learning rate, odnosno, brzina kojom optimizator prilagođava težine mreže. Prevelika vrednost može dovesti do nestabilnog treniranja, premala usporava konvergenciju.
- **hidden_layers**: Lista koja definiše broj i veličinu skrivenih slojeva u mreži. Više slojeva ili veći broj neurona može povećati kapacitet modela, ali i rizik od prenaučenosti.
- **dropout**: Procenat neurona koji se nasumično isključuje tokom treniranja radi regularizacije. Pomaže u smanjenju prenaučenosti.
- **batch_size**: Broj uzoraka koji se obrađuje u jednom prolazu optimizatora. Manji batch smanjuje memorijski zahtev i može dodati varijansu u treniranju, veći batch može ubrzati treniranje ali zahteva više memorije.

Svaka kombinacija parametara predstavlja jedan eksperiment. Rezultati (MSE, RMSE, MAE, R²) se čuvaju i rangiraju po R². Omogućava identifikaciju najbolje konfiguracije za finalno treniranje modela.

Modul `grid_search` omogućava automatsko testiranje različitih kombinacija hiperparametara nad modelom kroz parametre navedene u konfiguraciji.

In [18]:
def create_grid_parameters():
    """
    Define the hyperparameter grid for experimentation.
    """
    return {
        'pca_components': [20, 25, 30, 35, 40],
        'lr': [0.001, 0.01, 0.02, 0.03],
        'hidden_layers': [
            [128, 64], 
            [256, 128, 64],
            [512, 256, 128],
            [512, 256, 128, 64]
        ],
        'dropout': [0.0, 0.1],
        'batch_size': [32, 64, 96]
    }

### Rezultati eksperimenata i zaključci

**Rezulati eksperimenata** rangirani su po vrednostima R² metrike. Rezultati svih eksperimenata (svih kombinacija) sadržani su u
`grid_search_result.txt` zajedno sa vrednostima parametara korišćenih u okviru eksperimenta.
Tri najbolje konfiguracije:
- Rank 1 (R²: 0.9000):
  - PCA Components: 25
  - Learning Rate: 0.02
  - Hidden Layers: [512, 256, 128]
  - Dropout: 0.0
  - Batch Size: 64
  - RMSE: 27716.884765625
  - MAE: 18201.21484375

- Rank 2 (R²: 0.8990):
  - PCA Components: 25
  - Learning Rate: 0.03
  - Hidden Layers: [512, 256, 128, 64]
  - Dropout: 0.0
  - Batch Size: 96
  - RMSE: 27858.630859375
  - MAE: 19455.927734375

- Rank 3 (R²: 0.8979):
  - Experiment ID: 451
  - PCA Components: 40
  - Learning Rate: 0.02
  - Hidden Layers: [512, 256, 128, 64]
  - Dropout: 0.0
  - Batch Size: 32
  - RMSE: 28004.130859375
  - MAE: 18851.828125

**Analiza i zaključci** na osnovu vrednosti parametara:
- **PCA komponente**: Najbolje performanse su uglavnom postignute sa 25 PCA komponenti, što sugeriše da smanjenje dimenzionalnosti do ovog broja zadržava ključne informacije za predikciju. Veći broj komponenti (npr. 40) ne donosi značajno poboljšanje, što implicira da postoji višak informacija koji nije koristan za model.
- **Learning rate**: Optimalni LR je oko 0.02, iako i 0.01 i 0.03 daju solidne rezultate.
- **Hidden layers**: Najefikasnija arhitektura je [512, 256, 128] ili [512, 256, 128, 64], što pokazuje da mreža sa 3-4 skrivena sloja i postepeno smanjenim brojem neurona dobro modeluje kompleksnost podataka. Veće ili previše male mreže ne daju značajnu prednost.
- **Dropout**: Najbolji modeli imaju dropout = 0.0, što sugeriše da regularizacija kroz dropout nije bila neophodna za ovaj dataset i mrežu (možda zbog dovoljno velikog broja primera i PCA smanjenja dimenzionalnosti).
- **Batch size**: 64 je česta optimalna vrednost u modelima sa najboljim preformansama, ali i 32 i 96 daju slične rezultate. Ovo znači da model nije previše osetljiv na veličinu batch-a, što daje fleksibilnost pri treniranju.

Model je stabilan: razlike između top 20 eksperimenata u R² su male (0.900 → 0.888), što znači da je mreža relativno robusna na promene hiperparametara. PCA, learning rate i broj slojeva su najvažniji parametri koji utiču na performanse. Dropout i batch size imaju manji uticaj u ovom slučaju.